In [1]:
import pandas as pd
import numpy as np
import eng_to_ipa as ipa
import panphon as panphon

In [2]:
chinesedata = pd.read_csv('chineserace.csv', parse_dates = True)

In [3]:
import multiprocessing

def _apply(pool_map_args):
    df = pool_map_args[0]
    func = pool_map_args[1]
    args = pool_map_args[2]
#     print(args)
    
    return df.apply(func,axis = 1, args = args)

def parallel_apply(df, func, args):
    #指定主機有多少CPU，並把df 平均分配size
    num_processes = 10
    chunk_size = int(df.shape[0]/num_processes)

    #將df 拆解成 num_processes 個 chunk 的df list
    chunks = [df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]
    pool_map_args = [(df, func, args) for df in chunks]
    pool = multiprocessing.Pool(processes = num_processes)
    result = pool.map(_apply, pool_map_args)
    result = pd.concat([i for i in result])
    pool.terminate()
    pool.join()
    return result

In [4]:
def convert(row):
    return ipa.convert(row['First Name'])

In [5]:
chinesedata['IPA Name'] = parallel_apply(chinesedata, convert, args = ())

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [6]:
# SAVE IPA DATA INTO CSV FILE
# chinesedata.to_csv('withIPAName.csv', index = False)

# READ IPA DATA
IPAdata = pd.read_csv('withIPAName.csv', parse_dates = True)
lastnamelist = pd.read_csv('LastNameList.csv', parse_dates = True)

In [7]:
target = lastnamelist['Last Name'].tolist()

In [8]:
chiEthicsdata = pd.merge(IPAdata, lastnamelist, on = 'Last Name', how = 'left')

In [9]:
chiEthicsdata = chiEthicsdata.fillna('')

In [10]:
def merge_on_last_name(row):
    last_name = row['Last Name']
    if(row['姓氏'] == ''):
        for target_last_name in target:
            if(target_last_name in last_name.split()):
                return lastnamelist[lastnamelist['Last Name'] == target_last_name]['姓氏'].tolist()[0]
                
    return row['姓氏']

In [11]:
def merge_on_ethnic(row):
    last_name = row['Last Name']
    if(row['Ethnic'] == ''):
        for target_last_name in target:
            if(target_last_name in last_name.split()):
                return lastnamelist[lastnamelist['Last Name'] == target_last_name]['Ethnic'].tolist()[0]
                
    return row['Ethnic']

In [12]:
chiEthicsdata['姓氏'] = parallel_apply(chiEthicsdata, merge_on_last_name, ())
chiEthicsdata['Ethnic'] = parallel_apply(chiEthicsdata, merge_on_ethnic, ())

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [13]:
chiEthicsdata['姓氏'].isnull().sum()

0

In [14]:
# SAVE 姓氏 DATA INTO CSV FILE
chiEthicsdata.to_csv('withChiEthics.csv', index = False)

# READ IPA DATA
chiEthicsdata = pd.read_csv('withChiEthics.csv', parse_dates = True)

In [15]:
# DROP COLUMN ''
chiEthicsdata = chiEthicsdata.drop(columns = ['Degree','Intake Date','Intake Year','Name','Race'])

In [16]:
chiEthicsdata.head()

,Last Name,First Name,DTitle,Year,Age,IPA Name,姓氏,Ethnic
0,TEOH,HAN SEN,BACHELOR,1977.0,42.0,hɑn sɛn,張,Hokkien/Teowchew
1,LIM,YOKE FOONG,BACHELOR,1977.0,42.0,joʊk fuŋ,林,Hokkien
2,ANG,SOO SAN,BACHELOR,1977.0,42.0,su sæn,洪,Teochew
3,CALVIN TAN,KIAN SIN,BACHELOR,1977.0,42.0,ˈkiən sɪn,陳,Hokkien
4,CHAN,KHAN THAI,BACHELOR,1977.0,42.0,kɑn taɪ,陳,Cantonese
